<a href="https://colab.research.google.com/github/WanzeeCho/LG_CNS_ML/blob/main/Solving_A_Simple_Classification_Problem_with_Python_%E2%80%94_Fruits_Lovers%E2%80%99_Edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data**

과일 데이터 세트는 University of Edinburgh의 Iain Murray 박사가 만들었습니다. 

그는 다양한 종류의 오렌지, 레몬, 사과 수십 개를 사서 그 측정치를 표에 직접 기록했습니다. 

그리고 나서 University of Michigan의 교수들이 과일 데이터를 약간 형식화했으며

 https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/fruit_data_with_colors.txt 
 
 에서 다운로드할 수 있습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
fruits = pd.read_table('/content/drive/MyDrive/fruit_data_with_colors.txt')
fruits.head()

데이터 세트의 각 행은 테이블의 열에 있는 여러 기능으로 표시되는 과일 하나를 나타냅니다.

데이터 세트에는 59개의 과일과 7개의 Feature가 있습니다.

In [ ]:
print(fruits.shape)

데이터 세트에는 네 가지 유형의 과일이 있습니다.

In [ ]:
print(fruits['fruit_name'].unique())

데이터는 귤을 제외하고 꽤 균형이 잡혀 있습니다.

In [ ]:
print(fruits.groupby('fruit_name').size())

In [ ]:
import seaborn as sns
sns.countplot(x=fruits['fruit_name'],label="Count")
plt.show()

**Visualization**

각 숫자 변수의 Box Plot을 사용하면 입력 변수의 분포를 보다 명확하게 파악할 수 있습니다.

In [ ]:
fruits.drop('fruit_label', axis=1).plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False, figsize=(9,9), 
                                        title='Box Plot for each input variable')
plt.savefig('fruits_box')
plt.show()

일부 특성 쌍은 상관 관계가 있습니다(질량 및 너비).

In [ ]:
from pandas.plotting import scatter_matrix
from matplotlib import cm
feature_names = ['mass', 'width', 'height', 'color_score']
X = fruits[feature_names]
y = fruits['fruit_label']
cmap = cm.get_cmap('gnuplot')
scatter = scatter_matrix(X, c = y, marker = 'o', s=40, hist_kwds={'bins':15}, figsize=(9,9), cmap = cmap)
plt.suptitle('Scatter-matrix for each input variable')
plt.savefig('fruits_scatter_matrix')

데이터 스케일링이란 데이터 전처리 과정 중의 하나입니다.

피처(feature)들마다 데이터값의 범위가 다 제각각이기 때문에 범위 차이가 클 경우 데이터를 갖고 모델을 학습할 때 0으로 수렴하거나 무한으로 발산할 수 있습니다.

따라서 데이터 스케일링을 통해 모든 피처들의 데이터 분포나 범위를 동일하게 조정해줘야 합니다.

**Create Training and Test Sets and Apply Scaling**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

MinMaxScaler는 모든 피처들이 0과 1사이의 데이터값을 갖도록 만들어줍니다.

즉, 피처별로 최솟값은 0이 되고, 최댓값은 1이 되는 것이죠.

데이터가 2차원인 경우, 모든 데이터는 x, y 축의 0과 1 사이에 존재하게 됩니다.

**Build Models**

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.2f}'
     .format(lda.score(X_test, y_test)))

Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))

Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'
     .format(svm.score(X_test, y_test)))

KNN 알고리즘이 제일 정확도가 높은 모델이었습니다. Confusion Matrix를 확인해봅시다.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

**Plot the Decision Boundary of the k-NN Classifier**

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import matplotlib.patches as mpatches
import numpy as np

X = fruits[['mass', 'width', 'height', 'color_score']]
y = fruits['fruit_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

def plot_fruit_knn(X, y, n_neighbors, weights):
  X_mat = X[['height', 'width']].to_numpy()
  y_mat = y.to_numpy()
# Create color maps
  cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF','#AFAFAF'])
  cmap_bold  = ListedColormap(['#FF0000', '#00FF00', '#0000FF','#AFAFAF'])
  clf = KNeighborsClassifier(n_neighbors, weights=weights)
  clf.fit(X_mat, y_mat)

# Plot the decision boundary by assigning a color in the color map
# to each mesh point.
  
  mesh_step_size = .01  # step size in the mesh
  plot_symbol_size = 50
  
  x_min, x_max = X_mat[:, 0].min() - 1, X_mat[:, 0].max() + 1
  y_min, y_max = X_mat[:, 1].min() - 1, X_mat[:, 1].max() + 1
  xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step_size), np.arange(y_min, y_max, mesh_step_size))
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
# Put the result into a color plot
  Z = Z.reshape(xx.shape)
  plt.figure()
  plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
# Plot training points
  plt.scatter(X_mat[:, 0], X_mat[:, 1], s=plot_symbol_size, c=y, cmap=cmap_bold, edgecolor = 'black')
  plt.xlim(xx.min(), xx.max())
  plt.ylim(yy.min(), yy.max())

  patch0 = mpatches.Patch(color='#FF0000', label='apple')
  patch1 = mpatches.Patch(color='#00FF00', label='mandarin')
  patch2 = mpatches.Patch(color='#0000FF', label='orange')
  patch3 = mpatches.Patch(color='#AFAFAF', label='lemon')

  plt.legend(handles=[patch0, patch1, patch2, patch3])

  plt.xlabel('height (cm)')
  plt.ylabel('width (cm)')
  plt.title("4-Class classification (k = %i, weights = %s)" % (n_neighbors, weights))   
  plt.show()

plot_fruit_knn(X_train, y_train, 5, 'uniform')

In [ ]:
k_range = range(1, 20)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20])

이 데이터셋의 경우 k=5일 때 가장 높은 정확도를 얻습니다.